In [7]:
from environment.environment import CameraResolution, WarehouseBrawl
from environment.agent import RandomAgent, UserInputAgent, ConstantAgent, run_real_time_match

In [12]:
# Run a match against user input (Out of the box Agent options include: UserInputAgent, ConstantAgent, RandomAgent and more in environment/agent.py
agent_1 = UserInputAgent()
agent_2 = RandomAgent()
agent_3 = ConstantAgent()
max_timesteps = 30*90
run_real_time_match(agent_1, agent_2)

Failed to initialize pygame mixer
Obs space [-1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1] [1, 1, 1, 1, 1, 1, 1, 2, 12, 1, 1, 1, 1, 3, 11, 2, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 12, 1, 1, 1, 1, 3, 11, 2, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1]
Action space [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Ground is rendered
Ground is rendered
Stage is rendered


100%|██████████| 3/3 [00:00<00:00, 17.58it/s]


collided Hammer, True, True


MatchStats(match_time=47.3, player1=PlayerStats(damage_taken=37, damage_done=0, lives_left=3), player2=PlayerStats(damage_taken=0, damage_done=37, lives_left=0), player1_result=<Result.WIN: 1>)

In [11]:
# The following is a dissection of what happens when run_match is called
# env is initialized and initial observations are taken
env = WarehouseBrawl(resolution=CameraResolution.LOW, train_mode=False)
observations, info = env.reset()
obs_1 = observations[0]
obs_2 = observations[1]


platform1 = env.objects['platform1']

# On each tick, platform1 is moved, actions from agents (in this case, sampling from same agent) are taken, and the environment is stepped.
platform1.physics_process(0.05)
full_action = {
    0: agent_2.predict(obs_1),
    1: agent_2.predict(obs_2),
}

observations, rewards, terminated, truncated, info = env.step(full_action)
obs_1 = observations[0]
obs_2 = observations[1]

print("Observations and actions below")
print(f'{obs_1=}')
print(f'{full_action=}')


Obs space [-1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1] [1, 1, 1, 1, 1, 1, 1, 2, 12, 1, 1, 1, 1, 3, 11, 2, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 12, 1, 1, 1, 1, 3, 11, 2, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1]
Action space [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Observations and actions below
obs_1=array([ 5.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  7.        ,  0.        ,
       10.        ,  0.        ,  0.        ,  3.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.02427733,  1.02427733,
       -0.72831998,  0.7